In [34]:
import pickle
import gzip
import numpy as np

In [23]:
with gzip.open('/Users/Peace/Desktop/replays/INDEX.pkl', 'rb') as infile:
    master_index = pickle.load(infile)

print(master_index.keys())

dict_keys(['ts2018-halite-3-gold-replays_replay-20181014-012557%2B0000-1539480252-40-40', 'ts2018-halite-3-gold-replays_replay-20181014-000934%2B0000-1539475042-40-40', 'ts2018-halite-3-gold-replays_replay-20181014-000923%2B0000-1539475556-64-64', 'ts2018-halite-3-gold-replays_replay-20181014-011645%2B0000-1539479573-64-64', 'ts2018-halite-3-gold-replays_replay-20181014-002359%2B0000-1539475808-56-56', 'ts2018-halite-3-gold-replays_replay-20181014-004707%2B0000-1539477901-56-56', 'ts2018-halite-3-gold-replays_replay-20181014-011440%2B0000-1539479587-32-32', 'ts2018-halite-3-gold-replays_replay-20181014-010659%2B0000-1539478386-56-56', 'ts2018-halite-3-gold-replays_replay-20181014-003759%2B0000-1539477308-64-64', 'ts2018-halite-3-gold-replays_replay-20181014-004959%2B0000-1539477921-64-64', 'ts2018-halite-3-gold-replays_replay-20181014-010104%2B0000-1539478034-56-56', 'ts2018-halite-3-gold-replays_replay-20181014-011304%2B0000-1539479486-56-56', 'ts2018-halite-3-gold-replays_replay-2018

In [30]:
keep = []
for rp in master_index:
    for p in master_index[rp]['players']:
        if 'Rachol' == p['name'].split(' ')[0]:
            keep.append(rp)
            break
print(len(keep))

13


In [10]:
path = '/Users/Peace/Desktop/replays/{}/{}'
day = keep[0].replace('ts2018-halite-3-gold-replays_replay-', '').split('-')[0]
path = path.format(day, keep[0])

# with zstd.open(, 'rb') as infile:
#     replay = pickle.load(infile)

In [9]:
from core.data_utils import Game

In [11]:
game = Game()
game.load_replay(path)

In [31]:
print(game.production.shape)
print(game.moves.shape)
print(game.generate.shape)
print(game.entities.shape)
print(game.energy.shape)
print(len(game.deposited))
print(game.factories.shape)
print(len(game.dropoffs))
print(len(game.events))

for p in game.meta_data['players']:
    if 'Rachol' == p['name'].split(' ')[0]:
        pid = p['player_id']
print(pid)

(425, 40, 40)
(425, 40, 40, 2)
(425, 2)
(425, 40, 40, 4)
(426, 2)
426
(2, 2)
425
426
0


In [41]:

map_shape = game.production.shape[1], game.production.shape[2] 
factories = np.zeros((*map_shape, 1), dtype=np.float32)
for fx, fy in game.factories:
    factories[fy, fx] = -1 # Assume all are enemies

# Then set for the player of interest
fx, fy = game.factories[pid]
factories[fy, fx] = 1




In [58]:
# normalize some of the arrays
production = (game.production - 500.)/1000. # Guessing on norm values for now

entities = game.entities.copy()
has_ship = entities > 0
has_ship_mask = np.sum(has_ship.astype(np.float32), -1)
my_ships = has_ship[:, :, :, pid].copy()

entities = (np.sum(entities, -1) - 500.)/1000.


has_ship = -1 * np.sum(has_ship.astype(np.float32), axis=-1)
#print(entities.shape)
#print(my_ships.shape)
has_ship[my_ships] = 1
entities *= has_ship_mask
#print(has_ship[100, 20:40, 20:40])
#print(entities[100, 20:40, 20:40])
has_ship = np.expand_dims(has_ship, -1)
entities = np.expand_dims(entities, -1)

In [61]:
dropoffs = game.dropoffs
has_dropoff = np.zeros((production.shape[0], *map_shape, 1), dtype=np.float32)
assert len(dropoffs) == len(production)
for ix, ds in enumerate(dropoffs):
    for d in ds:
        x, y, oid = d
        v = 1. if oid == pid else -1.
        has_dropoff[ix, y, x] = v
print(has_dropoff.shape)

(425, 40, 40, 1)


In [63]:
# factories need to be duplicated accross frames
factories = np.repeat(np.expand_dims(factories, 0), production.shape[0], 0)
print(factories.shape)

(425, 40, 40, 1)


In [66]:
production = np.expand_dims(production, -1)

In [67]:
frames = np.concatenate([production, has_ship, entities, factories, has_dropoff], axis=-1)

In [69]:
frames.shape

(425, 40, 40, 5)

In [ ]:
# Note: 'no-moves' do not need explicit assignment since 0 means 'still'
# and the arrays are initialled with zero.

In [76]:
game.moves[100, 20:40, 20:40, 0]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0,

In [74]:
np.squeeze(has_ship[100, 20:40, 20:40])

array([[-0., -0., -0., -0., -0., -0., -0., -1., -0.,  1., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -0., -0., -0., -0.,  1., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -0., -0., -0., -0.,  1.,  1., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -